In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'f:\\ProjectAI\\FaceEmotionRecognitionSystem'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model: Path
    data_path: Path
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    all_params: dict

In [5]:
from FacialExpressionRecognition.constants import *
from FacialExpressionRecognition.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = self.config.training.trained_model_path,
            data_path = self.config.training.dataset_path,
            mlflow_uri = "https://dagshub.com/nhut-nam/FaceEmotionRecognitionSystemTest.mlflow",
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
            all_params = self.params
        )
        return eval_config

In [ ]:
# import dagshub
# dagshub.init(repo_owner='nhut-nam', repo_name='FaceEmotionRecognitionSystemTest', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\namnh\miniconda3\envs\FER\lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=62a9032b-06fd-4aba-b4da-f6422aa4d5ff&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1263e033746c5d952f32e0594f60c301c4c5408dff2baf2f85a331c959db8277


[2025-09-28 18:48:25,842: INFO: _client]: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"


[2025-09-28 18:48:26,220: INFO: _client]: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"
[2025-09-28 18:48:26,638: INFO: _client]: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as nhut-nam

[2025-09-28 18:48:26,645: INFO: helpers]: Accessing as nhut-nam
[2025-09-28 18:48:27,131: INFO: _client]: HTTP Request: GET https://dagshub.com/api/v1/repos/nhut-nam/FaceEmotionRecognitionSystemTest "HTTP/1.1 200 OK"
[2025-09-28 18:48:27,562: INFO: _client]: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Initialized MLflow to track repo "nhut-nam/FaceEmotionRecognitionSystemTest"

[2025-09-28 18:48:27,566: INFO: helpers]: Initialized MLflow to track repo "nhut-nam/FaceEmotionRecognitionSystemTest"


Repository nhut-nam/FaceEmotionRecognitionSystemTest initialized!

[2025-09-28 18:48:27,568: INFO: helpers]: Repository nhut-nam/FaceEmotionRecognitionSystemTest initialized!


In [11]:
import torch
import mlflow
from pathlib import Path
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision
from torchvision.datasets import ImageFolder
from urllib.parse import urlparse
import mlflow.pytorch
from FacialExpressionRecognition.models.resnet34 import get_resnet34_model

In [ ]:
class ModelEvaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def evaluate(self):
        # Load model
        self.model = self.load_model(model_path=self.config.path_of_model, params=self.config.all_params)

        # Data transformations
        data_transforms = transforms.Compose([
            transforms.Resize((self.config.params_image_size[0], self.config.params_image_size[1])),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # Load dataset
        train_dataset = ImageFolder(root=self.config.data_path + "/FER2013/train", transform=data_transforms)
        train_dataloader = DataLoader(train_dataset, batch_size=self.config.params_batch_size, shuffle=True)

        val_dataset = ImageFolder(root=self.config.data_path + "/FER2013/val", transform=data_transforms)
        val_dataloader = DataLoader(val_dataset, batch_size=self.config.params_batch_size, shuffle=False)

        test_dataset = ImageFolder(root=self.config.data_path + "/FER2013/test", transform=data_transforms)
        test_dataloader = DataLoader(test_dataset, batch_size=self.config.params_batch_size, shuffle=False)

        # Evaluate model
        train_accuracy, train_loss = self.evaluate_score(train_dataloader)
        val_accuracy, val_loss = self.evaluate_score(val_dataloader)
        test_accuracy, test_loss = self.evaluate_score(test_dataloader)

        print(f'Accuracy of the model on the training dataset: {train_accuracy:.2f}%')
        print(f'Accuracy of the model on the validation dataset: {val_accuracy:.2f}%')
        print(f'Accuracy of the model on the test dataset: {test_accuracy:.2f}%')

        self.metrics = {
            "train_accuracy": train_accuracy,
            "train_loss": train_loss,
            "val_accuracy": val_accuracy,
            "val_loss": val_loss,
            "test_accuracy": test_accuracy,
            "test_loss": test_loss
        }

        self.save_score()

    def evaluate_score(self, data_loader):
        correct = 0
        total = 0
        losses = []
        criterion = torch.nn.CrossEntropyLoss()
        with torch.no_grad():
            for images, labels in data_loader:
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                loss = criterion(outputs, labels)
                losses.append(loss.item())

        accuracy = 100 * correct / total
        return accuracy, sum(losses) / len(losses)

    @staticmethod
    def load_model(model_path, params):
        model = get_resnet34_model(pretrained=False, num_classes=params['NUM_CLASSES'])
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
        model.load_state_dict(checkpoint)
        
        model.eval()
        return model

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            for key, value in self.metrics.items():
                mlflow.log_metric(key, value)

            if tracking_url_type_store != "file":
                mlflow.pytorch.log_model(self.model, "model", registered_model_name=self.config.all_params['MODEL_NAME'])
            else:
                mlflow.pytorch.log_model(self.model, "model")

    def save_score(self):
        save_json(path=Path("scores.json"), data=self.metrics)

    

In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    model_eval = ModelEvaluation(config=eval_config)
    model_eval.evaluate()
    model_eval.log_into_mlflow()
except Exception as e:
    raise e

[2025-09-28 18:51:14,565: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-28 18:51:14,566: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-28 18:51:14,567: INFO: common]: Directory created at: artifacts


c:\Users\namnh\miniconda3\envs\FER\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\namnh\miniconda3\envs\FER\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Accuracy of the model on the training dataset: 22.27%
Accuracy of the model on the validation dataset: 23.82%
Accuracy of the model on the test dataset: 22.54%
[2025-09-28 18:52:46,289: INFO: common]: JSON file saved at: scores.json


2025/09/28 18:52:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'resnet34'.
2025/09/28 18:53:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: resnet34, version 1
Created version '1' of model 'resnet34'.


🏃 View run fortunate-carp-296 at: https://dagshub.com/nhut-nam/FaceEmotionRecognitionSystemTest.mlflow/#/experiments/0/runs/edcc7675dcbb42af81f804983d63ba27
🧪 View experiment at: https://dagshub.com/nhut-nam/FaceEmotionRecognitionSystemTest.mlflow/#/experiments/0
